### Donor-level stats in cellxgene data portal

Following [sfaira tutorial](https://github.com/theislab/sfaira_tutorials/blob/master/tutorials/cellxgene_download.ipynb)

In [1]:
import sfaira
import scanpy as sc
import pandas as pd
import numpy as np
import anndata
import os
import glob


In [2]:
data_path = os.path.join("/nfs/team205/ed6/data/", "sfaira_data")
cache_path = os.path.join("/nfs/team205/ed6/data/", "sfaira_cache")
meta_path = os.path.join("/nfs/team205/ed6/data/", "sfaira_meta")
if not os.path.exists(data_path):
    os.mkdir(data_path)
if not os.path.exists(cache_path):
    os.mkdir(cache_path)
if not os.path.exists(meta_path):
    os.mkdir(meta_path)

In [3]:
sfaira.settings.cachedir_base = cache_path

## Interact w database

In [47]:
dsg = sfaira.data.dataloaders.databases.DatasetSuperGroupDatabases(
    data_path=data_path, cache_path=cache_path, meta_path=meta_path, cache_metadata=True)

Ontology <class 'sfaira.versions.metadata.base.OntologyUberonLifecyclestage'> is not a DAG, treat child-parent reasoning with care.
Ontology <class 'sfaira.versions.metadata.base.OntologyMondo'> is not a DAG, treat child-parent reasoning with care.
Ontology <class 'sfaira.versions.metadata.base.OntologyUberon'> is not a DAG, treat child-parent reasoning with care.


ConnectionError: HTTPSConnectionPool(host='api.cellxgene.cziscience.com', port=443): Max retries exceeded with url: /dp/v1/collections/a238e9fa-2bdf-41df-8522-69046f99baff (Caused by ReadTimeoutError("HTTPSConnectionPool(host='api.cellxgene.cziscience.com', port=443): Read timed out. (read timeout=5)"))

In [48]:
## Subset to human collections
dsg.subset(key='organism', values=['Homo sapiens']) 

## Subset to scRNA-seq protocols (exclude spatial)
protocols = ["10x 3' transcription profiling", "10x 3' v1", "10x 3' v2",
       "10x 3' v3", "10x 5' transcription profiling", "10x 5' v1",
        'CEL-seq2', 'Drop-seq',
        'Smart-seq2', 
       'microwell-seq', 'sci-RNA-seq', 'single-cell RNA sequencing']
dsg.subset(key='assay_sc', values=protocols)

## Keep collections from a known primary tissue
target_collections = []
for k,v in dsg.datasets.items():
    if v.organ is not None:
        target_collections.append(k)
# dsg.subset(key="collection_id", values=target_collections)

NameError: name 'dsg' is not defined

In [ ]:
## Keep collections from published data
DOIs = []
for k in target_collections:
    DOIs.append(dsg.datasets[k].doi)

In [ ]:
target_collections = [target_collections[i] for i,d in enumerate(DOIs) if d[0].startswith('https://doi.org/')]

In [ ]:
dsg.subset(key="id", values=target_collections)

In [ ]:
len(dsg.datasets)

In [ ]:
# dsg.download()

In [ ]:
k = '0fdb6122-4600-40f0-a703-2da47cc7080d'
v = dsg.datasets[k]

In [48]:
v.load()

ValueError: adata of 0fdb6122-4600-40f0-a703-2da47cc7080d already loaded.

In [49]:
v.streamline_features(
        remove_gene_version=True,
        match_to_release={"Homo sapiens": "104"},#, "Mus musculus": "104"},
        subset_genes_to_type="protein_coding"
    )
v.streamline_metadata(keep_orginal_obs=True, keep_id_obs=True )

KeyError: 'ensembl'

In [ ]:
v.adata.obs

In [33]:
obs_to_keep = ['disease', 'sex', 'tissue', 'assay', 'ethnicity', 'development_stage']
donor_obs = v.adata.obs[obs_to_keep + [o + '_ontology_term_id' for o in obs_to_keep] + ["Patient"]].drop_duplicates()

In [56]:
help(re.match)

Help on function match in module re:

match(pattern, string, flags=0)
    Try to apply the pattern at the start of the string, returning
    a Match object, or None if no match was found.



In [75]:
get_putative_donor_obs_col(a)

array(['donor_id'], dtype=object)

In [68]:
a.obs.columns

Index(['n_counts_mrna', 'NUM.SNPS', 'NUM.READS', 'DROPLET.TYPE', 'BEST.GUESS',
       'percent_mito', 'pool', 'well', 'pool_well', 'batch', 'pool_clust',
       'freemux_cluster', 'donor_id', 'timepoint', 'respiratory_support_D0',
       'onset_to_D0_days', 'intubated_days', 'admission_to_discharge',
       'D0_to_death', 'race', 'consent', 'death', 'pulmonary_infection',
       'non_pulmonary_infection', 'leiden', 'onset_to_D0_days-as_float',
       'original_leiden', 'ct1', 'ct2', 'ct3', 'WBC_count1', 'WBC_count2',
       'WBC_count3', 'Lymphocyte_count', 'Monocyte_count', 'cell_group',
       'exclude_restricted', 'IFN_shared_score', 'IFN_beta_score',
       'IFN_gamma_score', 'days_from_start', 'COVID_status', 'admission_level',
       'respiratory_support', 'NIH_clinical', 'COVID_severity',
       'COVID_severity_merged', 'NIH_ordinal', 'tissue_ontology_term_id',
       'assay_ontology_term_id', 'disease_ontology_term_id',
       'development_stage_ontology_term_id', 'cell_type_on

In [22]:
os.listdir(v.data_dir)

['0fdb6122-4600-40f0-a703-2da47cc7080d.h5ad',
 '83b5e943-a1d5-4164-b3f2-f7a37f01b524.h5ad',
 'bdf69f8d-5a96-4d6f-a9f5-9ee0e33597b7.h5ad',
 'f7995301-7551-4e1d-8396-ffe3c9497ace.h5ad',
 'ed2b673b-0279-454a-998c-3eec361edf54.h5ad',
 '1252c5fb-945f-42d6-b1a8-8a3bd864384b.h5ad',
 '1062c0f2-2a44-4cf9-a7c8-b5ed58b4728d.h5ad',
 '9434b020-de42-43eb-bcc4-542b2be69015.h5ad',
 '65badd7a-9262-4fd1-9ce2-eb5dc0ca8039.h5ad']

In [ ]:
cellxgene_donor_obs = pd.DataFrame()
for k,v in dsg.datasets.items():
    ## Restrict to datasets with disease conditions
    if v.disease is None or v.disease != 'healthy':
        print(f"Loading {k}...")
        if v.adata is None:
            v.load()
        obs_to_keep = ['disease', 'sex', 'tissue', 'assay', 'ethnicity', 'development_stage']
        donor_obs = v.adata.obs[obs_to_keep + [o + '_ontology_term_id' for o in obs_to_keep]].drop_duplicates()
        donor_obs['dataset'] = k
        cellxgene_donor_obs = pd.concat([cellxgene_donor_obs, donor_obs])
        v.clear()

Loading edc8d3fe-153c-4e3d-8be0-2108d30f8d70...
Loading 2a498ace-872a-4935-984b-1afa70fd9886...
Loading d5c67a4e-a8d9-456d-a273-fa01adb1b308...
Loading c2a461b1-0c15-4047-9fcb-1f966fe55100...
Loading 1252c5fb-945f-42d6-b1a8-8a3bd864384b...
Loading bdf69f8d-5a96-4d6f-a9f5-9ee0e33597b7...
Loading 83b5e943-a1d5-4164-b3f2-f7a37f01b524...
Loading 9434b020-de42-43eb-bcc4-542b2be69015...
Loading ed2b673b-0279-454a-998c-3eec361edf54...
Loading f7995301-7551-4e1d-8396-ffe3c9497ace...
Loading 65badd7a-9262-4fd1-9ce2-eb5dc0ca8039...
Loading 0fdb6122-4600-40f0-a703-2da47cc7080d...
Loading 1062c0f2-2a44-4cf9-a7c8-b5ed58b4728d...
Loading 8e47ed12-c658-4252-b126-381df8d52a3d...
Loading 774c18c5-efa1-4dc5-9e5e-2c824bab2e34...
Loading 1c739a3e-c3f5-49d5-98e0-73975e751201...
Loading f15e263b-6544-46cb-a46e-e33ab7ce8347...
Loading 06b91002-4d3d-4d2e-8484-20c3b31e232c...
Loading 6c600df6-ddca-4628-a8bb-1d6de1e3f9b4...
Loading 75a881cf-5d88-46e2-bf9b-97e5cbc1bd56...
Loading 873ff933-4fda-4936-9a70-67df11af

### Read with scanpy

In [4]:
from anndata import AnnData
def get_putative_donor_obs_col(a: AnnData):
    return(a.obs.columns.values[a.obs.columns.str.contains("donor|patient", case=False)])

def get_donor_obs(data_path: str, 
                  c: str, 
                  d: str,
                 obs_to_keep = ['disease', 'sex', 'tissue', 'assay', 'ethnicity', 'development_stage']) -> pd.DataFrame:
    '''
    Params:
    -------
    - data_path: path to cellxgene data folder
    - c: collection ID
    - d: dataset .h5ad
    
    '''
    a = sc.read(os.path.join(data_path, c, d), 
                backed=True)
    if a.n_obs > 0:
        ## Find donor ID column
        donor_col = get_putative_donor_obs_col(a)
        print(donor_col)
    
        if len(donor_col) > 0:
            donor_obs = a.obs[obs_to_keep + [o + '_ontology_term_id' for o in obs_to_keep] + list(donor_col)].drop_duplicates()
            donor_obs['dataset'] = d.split('.h5ad')[0]
            donor_obs['collection'] = c
            return(donor_obs)
        else:
            return(None)
    else:
        return(None)

n_collections = len(os.listdir(data_path))
for i,c in enumerate(os.listdir(data_path)):
    print(f"Reading {c} ({i+1}/{n_collections})...")
    collection_donor_obs = pd.DataFrame()
    if not os.path.exists(os.path.join(data_path, c, 'donor_lvl_obs.csv')):
        for d in os.listdir(os.path.join(data_path, c)):
            donor_obs = get_donor_obs(data_path, c, d)
            if donor_obs is not None:
                collection_donor_obs = pd.concat([collection_donor_obs, donor_obs])
        collection_donor_obs.to_csv(os.path.join(data_path, c, 'donor_lvl_obs.csv'))



Reading 32f2fd23-ec74-486f-9544-e5b2f41725f5 (1/68)...
Reading a261413d-835b-4f1e-ab0c-dada55ea6afd (2/68)...
Reading 2d2e2acd-dade-489f-a2da-6c11aa654028 (3/68)...
Reading e02201d7-f49f-401f-baf0-1eb1406546c0 (4/68)...
Reading db468083-041c-41ca-8f6f-bf991a070adf (5/68)...
Reading 2b02dff7-e427-4cdc-96fb-c0f354c099aa (6/68)...
Reading 558385a4-b7b7-4eca-af0c-9e54d010e8dc (7/68)...
Reading ed9185e3-5b82-40c7-9824-b2141590c7f0 (8/68)...
Reading 180bff9c-c8a5-4539-b13b-ddbc00d643e6 (9/68)...
Reading 0434a9d4-85fd-4554-b8e3-cf6c582bb2fa (10/68)...
Reading fbc5881f-1ee3-4ffe-8095-35e15e1a08fc (11/68)...
Reading be21c2d1-2392-47d0-96fb-c625d115e0dc (12/68)...
Reading af893e86-8e9f-41f1-a474-ef05359b1fb7 (13/68)...
Reading 62ef75e4-cbea-454e-a0ce-998ec40223d3 (14/68)...
Reading b0cf0afa-ec40-4d65-b570-ed4ceacc6813 (15/68)...
Reading 120e86b4-1195-48c5-845b-b98054105eec (16/68)...
Reading 0a839c4b-10d0-4d64-9272-684c49a2c8ba (17/68)...
Reading c114c20f-1ef4-49a5-9c2e-d965787fb90c (18/68)...
R

In [17]:
obs_to_keep = ['disease', 'sex', 'tissue', 'assay', 'ethnicity', 'development_stage']

In [24]:
cellxgene_donor_obs = pd.DataFrame()

n_collections = len(os.listdir(data_path))
for i,c in enumerate(os.listdir(data_path)):
    collection_donor_obs = pd.read_csv(os.path.join(data_path, c, 'donor_lvl_obs.csv'))
    cellxgene_donor_obs = pd.concat([cellxgene_donor_obs, collection_donor_obs])

In [33]:
a = sc.read(os.path.join(data_path, c, d), 
                backed=True)

In [43]:
a.obs['donor_id']

barcode_index
AAACCTGAGAAGATTC-1-0     Rep_C_1025
AAACCTGAGACGCACA-1-0     ICC_C_0004
AAACCTGAGACTAAGT-1-0     Rep_C_1007
AAACCTGAGCCATCGC-1-0     Rep_C_1001
AAACCTGAGCCGATTT-1-0     Rep_C_1001
                            ...    
TTTGTCATCCTCAATT-1-49    Rep_C_1131
TTTGTCATCGTGACAT-1-49    Rep_C_1037
TTTGTCATCTCCAACC-1-49    Rep_C_1131
TTTGTCATCTCGTTTA-1-49    Rep_C_1043
TTTGTCATCTTCAACT-1-49    Rep_C_1154
Name: donor_id, Length: 600929, dtype: category
Categories (80, object): ['ICC-C-0134', 'ICC-C-0177', 'ICC-C-0666', 'ICC-C-1117', ..., 'Rep_C_1149', 'Rep_C_1151', 'Rep_C_1154', 'Rep_C_1161']

In [ ]:
os.listdir

In [31]:
cellxgene_donor_obs[cellxgene_donor_obs['collection'] == '62e8f058-9c37-48bc-9200-e767f318a8ec']

,Unnamed: 0,disease,sex,tissue,assay,ethnicity,development_stage,disease_ontology_term_id,sex_ontology_term_id,tissue_ontology_term_id,...,development_stage_ontology_term_id,dataset,collection,barcode,index,cell,cell_id,covid_index,cellId,barcode_index
0,RU1311A_T_1_165945547864806,small cell lung carcinoma,female,lung,10x 3' v3,European,77-year-old human stage,MONDO:0008433,PATO:0000383,UBERON:0002048,...,HsapDv:0000171,576f193c-75d0-4a11-bd25-8676587e6dc2,62e8f058-9c37-48bc-9200-e767f318a8ec,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RU1215_192110488599350,small cell lung carcinoma,female,pleural effusion,10x 3' v3,European,75-year-old human stage,MONDO:0008433,PATO:0000383,UBERON:0000175,...,HsapDv:0000169,576f193c-75d0-4a11-bd25-8676587e6dc2,62e8f058-9c37-48bc-9200-e767f318a8ec,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RU1057_T_231270811326245,lung adenocarcinoma,female,lung,10x 3' v2,European,67-year-old human stage,MONDO:0005061,PATO:0000383,UBERON:0002048,...,HsapDv:0000161,576f193c-75d0-4a11-bd25-8676587e6dc2,62e8f058-9c37-48bc-9200-e767f318a8ec,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RU1152_130751366121844,small cell lung carcinoma,male,lymph node,10x 3' v3,European,71-year-old human stage,MONDO:0008433,PATO:0000384,UBERON:0000029,...,HsapDv:0000165,576f193c-75d0-4a11-bd25-8676587e6dc2,62e8f058-9c37-48bc-9200-e767f318a8ec,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RU699_227364270922139,lung adenocarcinoma,female,adrenal gland,10x 3' v2,European,44-year-old human stage,MONDO:0005061,PATO:0000383,UBERON:0002369,...,HsapDv:0000138,576f193c-75d0-4a11-bd25-8676587e6dc2,62e8f058-9c37-48bc-9200-e767f318a8ec,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,RU1215_170677500798750,small cell lung carcinoma,female,pleural effusion,10x 3' v3,European,75-year-old human stage,MONDO:0008433,PATO:0000383,UBERON:0000175,...,HsapDv:0000169,f64e1be1-de15-4d27-8da4-82225cd4c035,62e8f058-9c37-48bc-9200-e767f318a8ec,NaN,NaN,NaN,NaN,NaN,NaN,NaN
373,RU1181B_192721179793635,small cell lung carcinoma,male,lung,10x 3' v3,European,62-year-old human stage,MONDO:0008433,PATO:0000384,UBERON:0002048,...,HsapDv:0000156,f64e1be1-de15-4d27-8da4-82225cd4c035,62e8f058-9c37-48bc-9200-e767f318a8ec,NaN,NaN,NaN,NaN,NaN,NaN,NaN
374,RU653_T_235120042097909,lung adenocarcinoma,female,lung,10x 3' v2,Asian,57-year-old human stage,MONDO:0005061,PATO:0000383,UBERON:0002048,...,HsapDv:0000151,f64e1be1-de15-4d27-8da4-82225cd4c035,62e8f058-9c37-48bc-9200-e767f318a8ec,NaN,NaN,NaN,NaN,NaN,NaN,NaN
375,RU684_T_164631959235301,lung adenocarcinoma,female,lung,10x 3' v2,European,70-year-old human stage,MONDO:0005061,PATO:0000383,UBERON:0002048,...,HsapDv:0000164,f64e1be1-de15-4d27-8da4-82225cd4c035,62e8f058-9c37-48bc-9200-e767f318a8ec,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
cellxgene_donor_obs

,disease,sex,tissue,assay,ethnicity,development_stage
0,normal,female,uterus,10x 3' v2,unknown,37-year-old human stage
1,normal,female,uterus,10x 3' v2,unknown,23-year-old human stage
2,normal,female,uterus,10x 3' v3,unknown,29-year-old human stage
3,normal,female,uterus,10x 3' v3,unknown,26-year-old human stage
4,normal,female,uterus,10x 3' v3,unknown,human early adulthood stage
...,...,...,...,...,...,...
4,COVID-19,male,blood,10x 5' transcription profiling,Oceanian,human adult stage
5,COVID-19,female,blood,10x 5' transcription profiling,unknown,human adult stage
6,COVID-19,female,blood,10x 5' transcription profiling,Asian,human adult stage
7,COVID-19,female,blood,10x 5' transcription profiling,African American,human adult stage


In [ ]:
cellxgene_donor_obs.to_csv(data_path + "/donor_lvl_metadata.csv")